# 🤖 AI Agent Example: Llama3 + RAG + Tool + ChromaDB
This notebook shows how to:
- Run a local AI agent using **llama3 via Ollama**
- Use **ChromaDB** to load RAG content
- Use **Open-Meteo API** as a tool
- Combine results using **LangChain's QA system**

In [ ]:
# Step 1: Install required packages
!pip install -q langchain chromadb requests llama-index

In [ ]:
# Step 2: Prepare and store local RAG data in ChromaDB
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
import os

# Save local tip documents
rag_docs = [
    "In Paris, it's best to carry an umbrella in spring.",
    "Always check the local metro schedule to avoid delays.",
    "Wear comfortable walking shoes when visiting tourist spots in Paris."
]

os.makedirs("rag_docs", exist_ok=True)
for i, text in enumerate(rag_docs):
    with open(f"rag_docs/doc_{i}.txt", "w") as f:
        f.write(text)

# Load and split
all_docs = []
for file in os.listdir("rag_docs"):
    loader = TextLoader(f"rag_docs/{file}")
    all_docs.extend(loader.load())

splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
splits = splitter.split_documents(all_docs)

# Embed and persist to Chroma
embedding = OllamaEmbeddings(model="llama3")
vectordb = Chroma.from_documents(splits, embedding, persist_directory="./chroma_db")
vectordb.persist()

In [ ]:
# Step 3: Define the weather tool using Open-Meteo API
import requests

def get_weather(city: str = "Paris"):
    response = requests.get(
        "https://api.open-meteo.com/v1/forecast",
        params={
            "latitude": 48.8566,
            "longitude": 2.3522,
            "current": "temperature_2m,wind_speed_10m",
            "temperature_unit": "celsius",
            "windspeed_unit": "kmh"
        }
    )
    if response.status_code != 200:
        return {"error": "Failed to fetch weather data."}
    return response.json().get("current", {})

In [ ]:
# Step 4: Run LLM + Tool + RAG Agent
from langchain.llms import Ollama
from langchain.chains import RetrievalQA

llm = Ollama(model="llama3")
retriever = Chroma(persist_directory="./chroma_db", embedding_function=embedding).as_retriever()
qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# Get weather data
weather = get_weather("Paris")
weather_text = (
    f"The current temperature in Paris is {weather.get('temperature_2m', '?')}°C "
    f"with wind speed {weather.get('wind_speed_10m', '?')} km/h."
)

query = f"Given this weather: {weather_text}, what travel tips do you have for Paris?"
response = qa.run(query)

print("🤖 AI Agent Response:\n")
print(response)